### Importing jdc so I can run the functions outside of the class cell.

In [ ]:
pip install jdc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Importing required libraries

In [ ]:
from __future__ import print_function, division
import numpy as np
from random import Random, random
import jdc

#### The State class. It keeps stack of the state, which is a list of 8 integers ranging from 1 to 8.

#### The init function saves the class properties to memory and allows the parent values and the generation number to be passed in.

#### If there are no parents, the state property initialises itself as a list of random numbers of length 8. If there are parents, they are assigned to the variables parent1 and parent2, set as the parents property, put through the crossover function and the mutate function is then ran with those arguments.

In [ ]:
class State(object):
    MUTATION_RATE = 0.4 # Play with this value, higher value means more chance of mutation
    def __init__(self, parents=None, generation = 1):
        r = Random()
        self._fitness = None
        self._probability = None
        self.generation = generation

        if parents==None:
            self.parents = None
            self.state = [r.randint(1,8) for y in range(8)]

        else:            
            parent1 = parents[0]
            parent2 = parents[1]
            self.parents = parents
            self.state = self.crossover(parent1, parent2)
            self.mutate()

#### This is measuring the fitness score, which depends on the number of collisions between two queens. 

#### The state is passed in from the class property to the fitness function. 

#### It counts the number of times a number in the list matches with another number in the list and divides it by 2 (as two queens would match with each other). It also counts the number of diagonal collisions divided by 2.

#### The fitness score is 28 (the maximum number of collisions) minus the sum of horizonal collisions + diagonal collisions.

#### It saves the fitness score to the fitness property.

#### A solution would have 0 collisions.

In [ ]:
%%add_to State
def fitness(self):                                                                                  
    if not self._fitness:
        state = self.state

        horizontal_collisions = sum([state.count(col)-1 for col in state])/2

        diagonal_collisions = 0
        for i, col in enumerate(state):
            for j, diagonal in enumerate(state):
                mod = abs(i-j)
                if mod > 0: #we don't want to count the current queen in a collision with herself
                    if diagonal + mod == col or diagonal - mod == col:
                        diagonal_collisions += 1
        diagonal_collisions /= 2 #we were counting the diagonal collisions double
        self._fitness = int(28 - (horizontal_collisions + diagonal_collisions))
    return self._fitness

#### This is giving each state a probability value based on it's fitness score. The better the fitness score, the higher the probability.

In [ ]:
%%add_to State
def probability(self, population):
    if not self._probability:
        self._probability = self.fitness() / sum([x.fitness() for x in population])
    return self._probability


#### The crossover function. This is calling in parent 1 and parent 2 and returning a child that has the values of a random split between then.

In [ ]:
%%add_to State
def crossover(self, parent1, parent2):
    r = Random()
    crossover_index = r.randint(0,8)
    left = parent1.state[0:crossover_index]
    
    right = parent2.state[crossover_index:8]
    left.extend(right)
    return left

#### The mutate function. For each state, each value has a random chance (depending on MUTATION_RATE) to change one of it's values to a random value between 1 and 8.

In [ ]:
%%add_to State
def mutate(self):
    r = Random()
    for i in range(len(self.state)):
        if random() < State.MUTATION_RATE:
            self.state[i] = r.randint(1,8)

#### A string representation of the state. This creates a chessboard visualisation of the list in self.state. 

In [ ]:
%%add_to State
def __str__(self):

    queen_symbol = '♛'
    blank_white_symbol = '▧'
    blank_black_symbol = '☐'

    # _state = np.zeros((8,8), dtype = np.uint8)
    # Create a blank board with alternating 0 and 1
    _i = np.sum(np.indices((8,8)), axis = 0)
    _state = np.where(_i % 2 == 0, 0, 1)
  
    for i, j in enumerate(self.state):
        _state[j - 1, i] = 2

    _repr = "Generation: " + str(self.generation) + "\nFitness: " + str(self.fitness()) + "\n" + str(self.state) +  "\n"
    for h in range(8):
        for w in range(8):
            _repr += ([blank_white_symbol, blank_black_symbol, queen_symbol][_state[h, w]] + " ")
        _repr += "\n"
    return _repr

#### Picking random states from the population. The states that are picked must have a lower probability score than the random function. Keeping the population as the same number.

In [ ]:
%%add_to State
def pickRandomByProbability(populationByProbability):
    i = 0
    selectedState = None
    while selectedState == None:
        current = populationByProbability[i]
        if current[0] <= random():
            return current[1]
        if i+1 <= len(populationByProbability):
            i = 0
        else:
            i += 1

#### Creating a new population. The states that have a lower probability score (fitness / the sum of all fitness scores)  have an increased chance of becoming parents in the next population.

In [ ]:
%%add_to State
def generateNextPopulation(population, n):
    newPopulation = []
    while len(newPopulation) < n:
        populationByProbability = [(x.probability(population), x) for x in population]
        parent1 = pickRandomByProbability(populationByProbability)
        populationByProbability = [x for x in populationByProbability if x[1] != parent1]
        parent2 = pickRandomByProbability(populationByProbability)
        newPopulation.append(State((parent1, parent2), generation = parent1.generation + 1))
    return newPopulation

#### The main function. This sets the initial values and creates a population list. When a fitness score of 28 has been reached, it shows the solution state. If none of the states in the population have a fitness score of 28, a new population is generated, and it shows the generation number, as well as the state with the max fitness score.

In [ ]:
%%add_to State
def main():
    populationSize = 100
    generation = 1
    population = [State(generation = generation) for x in range(populationSize)]
    
    while not 28 in [x.fitness() for x in population]:
        print("generation %d Max fitness: %d" % (generation, max([x.fitness() for x in population])))
        population = generateNextPopulation(population, populationSize)
        generation += 1

    for x in population:
        if x.fitness() == 28:            
            if x.parents is not None:
                print("Parents::")
                print(x.parents[0])
                print(x.parents[1])
            print("Solution state: ")
            print(x)

#### Runs the main function

In [ ]:
if __name__ == '__main__':
    main()

generation 1 Max fitness: 25
generation 2 Max fitness: 24
generation 3 Max fitness: 25
generation 4 Max fitness: 25
generation 5 Max fitness: 24
generation 6 Max fitness: 25
generation 7 Max fitness: 25
generation 8 Max fitness: 25
generation 9 Max fitness: 24
generation 10 Max fitness: 25
generation 11 Max fitness: 25
generation 12 Max fitness: 24
generation 13 Max fitness: 24
generation 14 Max fitness: 27
generation 15 Max fitness: 25
generation 16 Max fitness: 25
generation 17 Max fitness: 24
generation 18 Max fitness: 25
generation 19 Max fitness: 25
generation 20 Max fitness: 25
generation 21 Max fitness: 24
generation 22 Max fitness: 26
generation 23 Max fitness: 24
generation 24 Max fitness: 25
generation 25 Max fitness: 24
generation 26 Max fitness: 25
generation 27 Max fitness: 24
generation 28 Max fitness: 24
generation 29 Max fitness: 25
generation 30 Max fitness: 25
generation 31 Max fitness: 24
generation 32 Max fitness: 25
generation 33 Max fitness: 25
generation 34 Max f